#main.ipynb에서 사용할 모듈 clone


1.   preprocess.py

    *   save_pickle - utils.py에 저장해둔 make_id_file 메소드를 이용해 tokenizer로 학습데이터셋을 토큰화 시킨 뒤 pickle 라이브러리를 이용해 토큰화 된 데이터를저장.
    *   load_pickle - temp라는 빈 리스트에 pickle 파일을 열어 파일 내부의 데이터를저장해 리턴해줌.
    *   data2dataset - *'bert-base-unbased'* 토크나이저를 load하고 현재 작업공간에 data set이 없다면 save_pickle 메소드로 데이터셋을 저장해주고 data set이 있는 경우에는 load_pickle로 데이터를 불러와 train set과 valid set을 구분하여 utils.py에 저장해둔 SentimentDataset 클래스로 각 데이터셋을 선언. test set에 대한 데이터도 마찬가지로 utils.py에서 불러와 처리해준 뒤 최종적으로 train_dataset, dev_dataset, test_dataset, test_df를 리턴.


2.   utils.py 

    *   baseline 코드에 있던 make_id_file & make_id_file_test, SentimentDataset class, collate_fn_style, compute_acc을 저장해둔 모듈. 추가적으로 sklearn패키지에서 제공하는 train_test_split을 이용한 get_small_sample 메소드 추가. 


3.   data file


*   baseline에서 주어진 dataset 5가지와(sentiment_dev0 & 1, train 0 & 1, test_no_label)preprocess.py 에서 저장한 pkl파일들이 저장된 폴더.





참고 사이트

(pickle이란?)https://docs.python.org/ko/3/library/pickle.html



In [ ]:
# this was the first time i ever cried from a haircut .

["not so great food and service ."]

[let me give my opinion , that 's what this site is for .]

[the beer was nice and cold !]

[there are better happy hours and better beers all around mill !]

[it did n't matter of she is good at all other times .]

[found the place even though it is hard due to bad signage .]

[bottom line they over promise and under deliver .]

[they can thank you for the low rating .]

[the mechanics are very amateur as usual .]

[actually , just keep walking .]

[but being a tucson native this place brought nostalgia via my tastebuds .]

[if you are looking to walk out transformed this is your place !]

[his humor makes a routine visit entertaining .]

[this spot was my favorite indian restaurant .]

[tires , alignment , brakes and more .]

[for the record i am a good cook , i use seasoning !]

[sometimes it 's a simple cut , other times is more complicated color .]

[it is a half a day trip from phoenix area .]

[incredible , low price specials and the occasional groupon .]

[that comes with the smaller location though .]

[simply , there are far superior places to go for sushi .]
 
[tonight though i ordered to go and the food was just as good .]

[blue corn tacos with chicken were excellent .]

[its not cheap but you get good value here .]

In [ ]:
!git clone https://github.com/22eming/goorm_nlp_project_1.git

Cloning into 'goorm_nlp_project_1'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 65 (delta 24), reused 47 (delta 23), pack-reused 10
Unpacking objects: 100% (65/65), done.


git에서 해당 파일들이 있는 카테고리로 이동

In [ ]:
%cd /content/goorm_nlp_project_1

/content/goorm_nlp_project_1


##wandb install

In [ ]:
!pip install transformers wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.4 MB/s 
     |████████████████████████████████| 1.8 MB 38.4 MB/s 
     |████████████████████████████████| 6.6 MB 40.9 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 101 kB 12.7 MB/s 
     |████████████████████████████████| 147 kB 58.5 MB/s 
     |████████████████████████████████| 181 kB 50.8 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=da68e6aefa39272f89bd72a172b6a04cf796f40c241b827007f0939c1401ba90
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


##필요 패키지 및 라이브러리 import

In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict
import torch
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

from preprocess import data2dataset
from utils import collate_fn_style, collate_fn_style_test, compute_acc, get_small_sample

# parameters 사용 예시 
##상수

```
parameter_name:
    distribution: constant
    value: 2.71828

```

##범주

```
  parameter_name:
    distribution: categorical
    values:
      - elu
      - celu
```
##uniform
```
  parameter_name:
    distribution: uniform
    min: 0
    max: 1
```
##quniform
```
  parameter_name:
    distribution: q_uniform
    min: 0
    max: 256
    q: 1
```

##else

```
parameters: {          
        epochs: {
            values: [10, 20, 50]
        },
```

wandb sweep에 필요한 자료 config

In [ ]:
sweep_config = {
    'method': 'bayes', # grid, random, bayes 중 선택
    'metric': {
      'name': 'val_loss',   # 목표 변수
      'goal': 'minimize'    # 목표
    },
    'parameters': {         # 조절하고 싶은 파라미터 
        'epochs': {
            'values': [1, 2, 3]
        },
        'dropout': {
            'values': [0.3, 0.4, 0.5]
        },
        'weight_decay': {
            'values': [0.0005, 0.005, 0.05]
        },
        'learning_rate': {
            'values': [1e-3, 1e-4, 3e-4, 3e-5, 1e-5]
        },
    }
}

## wandb import 및 sweep 실행.
project_name에 프로젝트 이름을 넣어주셔야 해당 프로젝트에 sweep이 생성됩니다!

Sweep Guide(by 혁기 & 정환)
https://www.notion.so/AutoML-Guide-sweep-2d6ae4fd307544ee87a4eb5affd43bac


In [ ]:
import wandb
progect_name = "dongil_sweep_test"
sweep_id = wandb.sweep(sweep_config, entity="goorm_nlp_project_1", project=progect_name)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: vfzw2gho
Sweep URL: https://wandb.ai/goorm_nlp_project_1/dongil_sweep_test/sweeps/vfzw2gho


# 데이터셋 가져오기

In [ ]:
train_dataset, dev_dataset, test_dataset, test_df = data2dataset()
# 데이터세트, 가져올 비율
'''
    def get_small_sample(dataset, getting_size = 0.2):
        _, small_sample = train_test_split(dataset, test_size=getting_size, random_state=42)
        return small_sample
    
    train_test_split 은 sklearn 패키지에서 제공하는 메소드로 일반적으로 실무에서 train set과 test 셋에 대한 
    data를 따로 가져오지 않기 때문에 전체 데이터 셋을 해당 메소드로 나누어 많이 사용하게 됨.
    일반적으론 Train/Valid/Test 셋에 대해 6:2:2 정도의 비율을 사용하게 됨. 
    
    위에 사용된 get_small_sample의 경우 단순히 전체 데이터 셋 양을 줄여 적은양의 샘플로 학습하고자 하는 목적이므로
    test set으로 분류된 data정보만 small_sample 변수에 저장하여 사용.

    일반적인 예시)
    from sklearn.model_selection import train_test_split

    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle=True)
'''
train_dataset = get_small_sample(train_dataset, 0.2)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

# Train

In [ ]:
#sweep.agent()에 파라미터로 넣어 sweep을 실행하고자 기본 베이스라인 코드에서 메소드로 형태를 변환하여 사용.
def train():
  lowest_valid_loss = 9999. #새로운 loss값과의 비교를 통해 가장 작은 loss를 찾아야 하므로 최솟값을 충분히 크게 설정.

  # preprocess
  # config 변경 가능
  config_defaults = {
    'epochs': 2,
    'batch_size': 128,
    'weight_decay': 0.0005,
    'learning_rate': 1e-5,
    'optimizer': 'AdamW',
    'seed': 42,
    'tr_loss_check' : 50
  }
  '''
  sweep 과정에서 분할 된 실험 셋에 대해 각각 wandb 정보를 init해야하므로 wand.init을 train 함수 내부로 옮기고
  모델의 configuration을 wandb와 동일하게 설정.
  '''
  wandb.init(config=config_defaults)
  config = wandb.config
  '''
    모델에 데이터를 미니배치 형태로 넣기 수월하게 하기 위해 Pytorch의 데이터로더에 데이터를 넣어주는 작업이다.
    DataLoader에 대한 설명 대신 본인이 직접 보고 이해했던 블로그 주소 링크 첨부.(길어질수 있어서)
    https://sanghyu.tistory.com/90
  '''
  train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
  dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=config.batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)
  """

    학습 중 사용될 랜덤 변수들에 대해 seed를 정해주는 작업이다.
    seed를 설정하지 않으면 기본적으로 '시간'을 기준으로 내부 알고리즘에 따라 계속해서 난수를 생성하는데
    시간은 계속 흐르고 있기 때문에 난수를 생성하는 기준이 되는 seed또한 계속해서 바뀌게 된다.
    때문에 seed를 parameter로 지정하여 넘겨줌으로써 seed 값을 고정하여 같은 기준(seed)에서 난수가 생성되게끔 해주는 역할을 한다.

  """
  np.random.seed(config.seed)
  torch.manual_seed(config.seed)

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # 모델의 학습작업을 메소드를 통해 gpu가 사용 가능한 경우엔 gpu(cuda)에 올려 학습

  model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
  model.to(device) #모델을 위에서 설정한 device에 올림.

  model.train() #train 모드로 변경.

  optimizer = AdamW(model.parameters(), lr=config.learning_rate, weight_decay=config.weight_decay) #optimizer 설정

  # train()
  for epoch in range(config.epochs):
      with tqdm(train_loader, unit="batch") as tepoch: #tqdm -> 상태 진행률을 프로세스바를 통해 시간적으로 확인할 수 있는 이터레이터 객체를 반환.
          for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
              tepoch.set_description(f"Epoch {epoch}")
              #각 값들에 대한 input Tensor들 호출.
              input_ids = input_ids.to(device)
              attention_mask = attention_mask.to(device)
              token_type_ids = token_type_ids.to(device)
              position_ids = position_ids.to(device)
              labels = labels.to(device, dtype=torch.long)
              
              optimizer.zero_grad() 
              """
                pytorch의 경우 gradients값을 backward()를 통해 계산할때마다 기존 값에 계속 더해주는 방식으로 연산이
                진행되기 때문에 backpropagation 작업시마다 optimizer의 gradients를 0으로 초기화시켜주지 않으면 한번의
                iteration이 끝날 때마다 이전의 graidients값이 남아있어 학습이 제대로 진행되지 않습니다.
              """

              output = model(input_ids=input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            labels=labels)
              #model의 출력값에서의 loss를 backpropagation.
              loss = output.loss
              loss.backward()
              
              optimizer.step() #optimizer update

              tepoch.set_postfix(loss=loss.item()) #콘솔에 결과 출력용
              
              #중간중간 train set에 대한 acc와 로스를 출력하기 위한 부분. config에 check할 iteration 단위를 저장하여 사용.
              if iteration % config.tr_loss_check == 0:
                logits = output.logits
                batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                batch_labels = [int(example) for example in labels]
                acc = compute_acc(batch_predictions, batch_labels)
                wandb.log({"train_loss" : loss, "train_acc" : acc})
                
              if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                  # 한 epoch에서 5개의 시점으로 나누어 5번의 validation 진행.
                  with torch.no_grad():  
                      '''
                        validation 과정에서는 gradients를 다룰 필요가 없기 때문에 Pytorch 내부의 autograd engine을 꺼줌으로서
                        validation 과정에서의 연산 속도를 올리는게 주요 목적.
                      '''
                      model.eval() #validataion 모드로 변경.
                      valid_losses = []
                      predictions = []
                      target_labels = []
                      for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                  desc='Eval',
                                                                                                  position=1,
                                                                                                  leave=None):
                          #위와 마찬가지로 input Tensor값을 호출해 모델에 넣어 output 저장(line 110-120)
                          input_ids = input_ids.to(device)
                          attention_mask = attention_mask.to(device)
                          token_type_ids = token_type_ids.to(device)
                          position_ids = position_ids.to(device)
                          labels = labels.to(device, dtype=torch.long)

                          output = model(input_ids=input_ids,
                                        attention_mask=attention_mask,
                                        token_type_ids=token_type_ids,
                                        position_ids=position_ids,
                                        labels=labels)

                          logits = output.logits
                          loss = output.loss
                          valid_losses.append(loss.item())

                          batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                          batch_labels = [int(example) for example in labels]

                          predictions += batch_predictions
                          target_labels += batch_labels

                  acc = compute_acc(predictions, target_labels)
                  valid_loss = sum(valid_losses) / len(valid_losses) #전체 valid set에 대한 loss값을 계산.
                  wandb.log({"val_acc":acc, "val_loss" : valid_loss})
                  if lowest_valid_loss > valid_loss: #최적 시점의 모델을 저장하는 구문 baseline 코드에선 validation loss가 가장 낮은곳으로 기준을 설정.
                      print('Acc for model which have lower valid loss: ', acc)
                      torch.save(model.state_dict(), "./pytorch_model.bin")

In [ ]:
#위에 설정한 train 메소드를 넣어주어 설정한 여러 하이퍼 파라미터별로 나누어 sweep 실행.
wandb.agent(sweep_id, train)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions


Test:  88%|████████▊ | 14/16 [00:00<00:00, 20.11it/s]
                                                     

In [ ]:
test_df['Category'] = predictions
test_df.to_csv('submission.csv', index=False)